In [1]:
import pandas as pd
import numpy as np

### 字串處理

`Series` 物件提供了大量的字串處理方法，由於數量許多，因此 Pandas 使用了一個類似 名稱空間 的物件 `str` 來包裝這些字串相關的方法。例如下面的程式呼叫 `str.upper()` 將序列中的所有字母都轉為大寫：

In [2]:
s_abc = pd.Series(["a", "b", "c"])
print( s_abc.str.upper() )

0    A
1    B
2    C
dtype: object


Python 中包含兩種字串：`位元組字串` 和 `Unicode字串`。

透過 `str.decode()` 可以將 `位元組字串` 按照指定的編碼解碼為 `Unicode字串`。例如在 UTF-8 編碼中，一個中文字佔用三個字元，因此下面的 `s_utf8` 中的字串長度分別為 6, 9, 12。當呼叫 `str.decode()` 將其轉為 `Unicode字串` 的序列之後，其各個元素的長度為實際的文字個數。

`str.encode()` 可以把 `Unicode字串` 按照指定的編碼轉為 `位元組字串`，在常用的中文編碼 GB2312 中，一個中文字佔用兩個位元組，因此 `s_gb2312` 的元素長度分別為 4, 6, 8。

PS: `decode()` 可以理解成 將字串解碼成人類看的懂的字串。`encode()` 可以理解成 將字串編碼成機器看的懂的 bytes。

In [10]:
# s_utf8 = pd.Series(["北京", b"北京市", b"北京地區"])
s_utf8 = pd.Series([bytes("北京", 'utf-8'), bytes("北京市", 'utf-8'), bytes("北京地區", 'utf-8')])
s_unicode = s_utf8.str.decode("utf-8")
# s_gb2312 = s_unicode.str.encode("gb2312")
s_big5 = s_unicode.str.encode("big5")

# %C s_utf8.str.len(); s_unicode.str.len(); s_gb2312.str.len()
print( s_utf8.str.len() )
print("-"*20)
print( s_unicode.str.len() )
print("-"*20)
print( s_big5.str.len() )

0     6
1     9
2    12
dtype: int64
--------------------
0    2
1    3
2    4
dtype: int64
--------------------
0    4
1    6
2    8
dtype: int64


無論 `Series` 物件包含哪種字串物件，其 `dtype` 屬性都是 `object` ，因此無法根據它判斷字串型態。在處理文字資料時，需要格外注意字串的型態。

可以對 `str` 使用整數或切片索引，相當於對 `Series` 物件中的每個元素進行索引運算，例如：

In [12]:
print( s_unicode.str[:2] )

0    北京
1    北京
2    北京
dtype: object


字串序列與字串一樣，支援加法和乘法運算，例如：

In [13]:
print( s_unicode + u"-" + s_abc * 2 )

0      北京-aa
1     北京市-bb
2    北京地區-cc
dtype: object


也可以使用 `str.cat()` 連接兩個字串序列的對應元素：

In [6]:
print s_unicode.str.cat(s_abc, sep="-")

0      北京-a
1     北京市-b
2    北京地區-c
dtype: object


呼叫 `astype()` 方法可以對 `Series` 物件中的所有元素進行型態轉換，例如下面將整數序列轉為字串序列：

In [15]:
print( s_unicode.str.len().astype('unicode') )

0    2
1    3
2    4
dtype: object


`str` 中的有些方法可以對元素型態為清單的 `Series` 物件進行處理，例如下面呼叫 `str.split()` 將 `s` 中的每個字串使用字元 `"|"` 分隔，所得到的結果 `s_list` 的元素型態為清單。然後呼叫它的 `str.join()` 方法以逗號連接每個清單的元素：

In [16]:
s = pd.Series(["a|bc|de", "x|xyz|yz"])
s_list = s.str.split("|")
s_comma = s_list.str.join(",")
# %C s; s_list; s_comma
print(s)
print("-"*20)
print(s_list)
print("-"*20)
print(s_comma)

0     a|bc|de
1    x|xyz|yz
dtype: object
--------------------
0     [a, bc, de]
1    [x, xyz, yz]
dtype: object
--------------------
0     a,bc,de
1    x,xyz,yz
dtype: object


對字串序列進行處理時，經常會獲得元素型態為清單的序列。Pandas 沒有提供處理這種序列的方法，不過可以透過 `str[]` 取得其中的元素：

In [17]:
s_list.str[1]

0     bc
1    xyz
dtype: object

或將其轉為嵌套列表，然後再轉為 `DataFrame` 物件：

In [18]:
print( pd.DataFrame(s_list.tolist(), columns=["A", "B", "C"]) )

   A    B   C
0  a   bc  de
1  x  xyz  yz


Pandas 還提供了一些正規表示法相關的方法。例如使用其中的 `str.extract()` 可以從字串序列中取出需要的部分，獲得 `DataFrame` 物件。下面的實例中，`df_extract1` 對應的正規表示法包含三個未命名的組，因此其結果包含三個自動命名的列。而 `df_extract2` 對應的正規表示法包含兩個命名組，因些其列名為組名。

In [19]:
df_extract1 = s.str.extract(r"(\w+)\|(\w+)\|(\w+)")
df_extract2 = s.str.extract(r"(?P<A>\w+)\|(?P<B>\w+)|")
# %C df_extract1; df_extract2
print(df_extract1)
print("-"*20)
print(df_extract2)

   0    1   2
0  a   bc  de
1  x  xyz  yz
--------------------
   A    B
0  a   bc
1  x  xyz


在處理資料時，經常會遇到這種以特定分隔符號分隔關鍵字的資料，例如下面的資料可以用於表示有方向圖，其第一列為邊的起點、第二列為以`"|"`分隔的多個終點。下面使用 `read_csv()` 讀取該資料，獲得一個兩列的 `DataFrame` 物件：

In [21]:
import io
text = b"""A, B|C|D
B, E|F
C, A
D, B|C
"""

df = pd.read_csv(io.BytesIO(text), skipinitialspace=True, header=None)
print( df )

   0      1
0  A  B|C|D
1  B    E|F
2  C      A
3  D    B|C


可以使用下面的程式將上述資料轉為每行對應一筆邊的資料。

❶ `nodes` 是一個元素型態為清單的 `Series` 物件。

❸ 呼叫 Numpy 陣列的 `repeat()` 方法將第一列資料重複對應的次數。由於 `repeat()` 只能接受 32 位元整數，而 `str.len()` 傳回的是 64 位元整數，因此還需要進行型態轉換。

❸ 將嵌套列表平坦化，轉為一維陣列。

In [22]:
nodes = df[1].str.split("|") #❶
from_node = df[0].values.repeat(nodes.str.len().astype(np.int32)) #❷
to_node = np.concatenate(nodes) #❸

print( pd.DataFrame({"from_node":from_node, "to_node":to_node}) )

  from_node to_node
0         A       B
1         A       C
2         A       D
3         B       E
4         B       F
5         C       A
6         D       B
7         D       C


還可以把原始資料的第二列看作一列資料的標籤，為了後續的資料分析，通常使用 `str.get_dummies()` 將這種資料轉為布林 `DataFrame` 物件，每一列與一個標籤對應，元素值為 1 表示對應的行包含對應的標籤：

In [18]:
print( df[1].str.get_dummies(sep="|") )

   A  B  C  D  E  F
0  0  1  1  1  0  0
1  0  0  0  0  1  1
2  1  0  0  0  0  0
3  0  1  1  0  0  0


當字串操作很難用向量化的字串方法表示時，可以使用 `map()` 函數，將針對每個元素運算的函數運用到整個序列之上：

In [23]:
df[1].map(lambda s:max(s.split("|")))

0    D
1    F
2    A
3    C
Name: 1, dtype: object

當用字串序列表示分類資訊時，其中會有大量相同的字串，將其轉為 `分類(Category)序列` 可以節省記憶體、加強運算效率。例如在下面的 `df_soil` 物件中，Contour, Depth, Gp 列都是表示分類的資料，因此有許多重複的字串。

In [24]:
df_soil = pd.read_csv("Soils.csv", usecols=[2, 3, 4, 6])
print( df_soil.dtypes )

Contour     object
Depth       object
Gp          object
pH         float64
dtype: object


In [28]:
df_soil.head()

,Contour,Depth,Gp,pH
0,Top,0-10,T0,5.40
1,Top,0-10,T0,5.65
2,Top,0-10,T0,5.14
3,Top,0-10,T0,5.14
4,Top,10-30,T1,5.14


下面循環呼叫 `astype("category")` 將這三列轉為分類列：

In [25]:
for col in ["Contour", "Depth", "Gp"]:
    df_soil[col] = df_soil[col].astype("category")
print( df_soil.dtypes )

Contour    category
Depth      category
Gp         category
pH          float64
dtype: object


與名稱空間物件 `str` 類似，元素型態為 `category` 的 `Series` 物件提供了名稱空間物件 `cat` ，其中儲存了與分類序列相關的各種屬性和方法。例如 `cat.categories` 是儲存所有分類的 `Index` 物件。

In [26]:
Gp = df_soil.Gp
print( Gp.cat.categories )

Index(['D0', 'D1', 'D3', 'D6', 'S0', 'S1', 'S3', 'S6', 'T0', 'T1', 'T3', 'T6'], dtype='object')


而 `cat.codes` 則是儲存索引的整數序列，元素型態為 `int8` ，因此一個元素用一個位元組表示。

In [29]:
# %C Gp.head(5); Gp.cat.codes.head(5)
print(Gp.head(5))
print("-"*20)
print(Gp.cat.codes.head(5))

0    T0
1    T0
2    T0
3    T0
4    T1
Name: Gp, dtype: category
Categories (12, object): ['D0', 'D1', 'D3', 'D6', ..., 'T0', 'T1', 'T3', 'T6']
--------------------
0    8
1    8
2    8
3    8
4    9
dtype: int8


分類資料有 `無序` 和 `有序` 兩種，無序分類中的不同分類無法比較大小，例如性別；有序分類則可以比較大小，例如年齡段。上面建立的三個分類列為無序分類，可以透過 `cat.as_ordered()` 和 `cat_as_unordered()` 在這兩種分類之間相互轉換。下面的程式透過 `cat.as_ordred()` 將深度分類列轉為有序分類，注意最後一行分類名之間使用 "<" 連接，表示是有序分類。

In [30]:
depth = df_soil.Depth
# %C depth.cat.as_ordered().head()
print( depth.cat.as_ordered().head() )

0     0-10
1     0-10
2     0-10
3     0-10
4    10-30
Name: Depth, dtype: category
Categories (4, object): ['0-10' < '10-30' < '30-60' < '60-90']


如果需要自訂分類中的順序，可以使用 `cat.reorder_categories()` 指定分類的順序：

In [31]:
contour = df_soil.Contour
categories = ["Top", "Slope", "Depression"]
# %C contour.cat.reorder_categories(categories, ordered=True).head()
print( contour.cat.reorder_categories(categories, ordered=True).head() )

0    Top
1    Top
2    Top
3    Top
4    Top
Name: Contour, dtype: category
Categories (3, object): ['Top' < 'Slope' < 'Depression']
